In [ ]:
# Data Manipulation and Preparation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Evaluation metrics and visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Utility imports
import itertools
import time
import csv
import os

# Set up GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load the preprocessed datasets
path_30min = ''
path_4h = ''

In [ ]:
df_30min = pd.read_csv(path_30min)
df_4h = pd.read_csv(path_4h)

In [ ]:
all_dfs = [df_30min, df_4h]

In [ ]:
# Convert timestamp columns to datetime
for df in all_dfs:
    df['open_time'] = pd.to_datetime(df['open_time'])
    df['close_time'] = pd.to_datetime(df['close_time'])

# Define feature sets
price_features = ['close', 'high', 'low', 'volume', 'quote_vol', 'count', 'buy_base', 'buy_quote']
diff_features = ['close_diff', 'high_diff', 'low_diff']

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, output_window_size, num_layers=1, dropout=0.0, activation_function=nn.ReLU):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = activation_function()
        self.fc2 = nn.Linear(dense_dim, output_dim * output_window_size)
        self.output_window_size = output_window_size

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x.view(-1, self.output_window_size, 1)

def create_sequences(data, target, input_window_size, output_window_size):
    sequences = []
    labels = []
    for i in range(len(data) - input_window_size - output_window_size + 1):
        seq = data[i:i + input_window_size]
        label = target[i + input_window_size:i + input_window_size + output_window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

def prepare_data(df, target_column, input_window_size, output_window_size, feature_columns):
    X = df[feature_columns].values
    y = df[target_column].values.reshape(-1, 1)
    
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, input_window_size, output_window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, input_window_size, output_window_size)

    return X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y

def plot_and_save_loss(training_loss, validation_loss, file_prefix, model_counter, start_epoch=3):
    plt.figure(figsize=(10, 6))
    plt.plot(range(start_epoch, len(training_loss)), training_loss[start_epoch:], label='Training Loss')
    plt.plot(range(start_epoch, len(validation_loss)), validation_loss[start_epoch:], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Model {model_counter} - Training and Validation Loss Over Epochs')
    plt.legend()
    plt.savefig(f'{file_prefix}_loss_plot_model_{model_counter}.png')
    plt.close()

    loss_df = pd.DataFrame({
        "Epoch": range(len(training_loss)),
        "Training Loss": training_loss,
        "Validation Loss": validation_loss
    })
    loss_file_path = f'{file_prefix}_losses_model_{model_counter}.csv'
    loss_df.to_csv(loss_file_path, index=False)

def save_predictions(y_train, y_train_pred, y_test, y_test_pred, model_counter, file_prefix):
    train_df = pd.DataFrame({"Actual": y_train.flatten(), f"Model_{model_counter}": y_train_pred.flatten()})
    test_df = pd.DataFrame({"Actual": y_test.flatten(), f"Model_{model_counter}": y_test_pred.flatten()})

    train_file_path = f"{file_prefix}_train_predictions_model_{model_counter}.csv"
    test_file_path = f"{file_prefix}_test_predictions_model_{model_counter}.csv"
    train_df.to_csv(train_file_path, index=False)
    test_df.to_csv(test_file_path, index=False)

In [ ]:
def train_and_evaluate(df, target_column, feature_columns, params, model_counter, file_prefix='results', use_early_stopping=False, plot_loss=False):
    result = {}
    try:
        window_size = params['window_size']
        output_window_size = params['output_window_size']
        lstm_dim = params['lstm_dim']
        dense_dim = params['dense_dim']
        num_layers = params['num_layers']
        dropout = params['dropout']
        lr = params['lr']
        batch_size = params['batch_size']
        num_epochs = params['num_epochs']
        optimizer_type = params['optimizer_type']
        patience = params['patience']
        activation_function = params['activation_function']

        X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, output_window_size, feature_columns)

        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).to(device)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

        input_dim = X_train_tensor.shape[2]
        output_dim = 1

        model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, output_window_size, num_layers, dropout, activation_function).to(device)

        criterion = nn.MSELoss()
        optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

        best_loss = float('inf')
        patience_counter = 0
        start_time = time.time()

        training_losses = []
        validation_losses = []

        for epoch in range(num_epochs):
            model.train()
            epoch_train_loss = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            epoch_train_loss /= len(train_loader)
            training_losses.append(epoch_train_loss)

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()

            val_loss /= len(test_loader)
            validation_losses.append(val_loss)

            if use_early_stopping:
                if val_loss < best_loss:
                    best_loss = val_loss
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print(f"Early stopping triggered at epoch {epoch}")
                        break

        training_time = time.time() - start_time

        model.eval()
        evaluation_start_time = time.time()
        with torch.no_grad():
            train_predictions = model(X_train_tensor).cpu().numpy()
            test_predictions = model(X_test_tensor).cpu().numpy()

            train_predictions_inverse = scaler_y.inverse_transform(train_predictions.reshape(-1, output_window_size))
            test_predictions_inverse = scaler_y.inverse_transform(test_predictions.reshape(-1, output_window_size))
            y_train_inverse = scaler_y.inverse_transform(y_train_seq.reshape(-1, output_window_size))
            y_test_inverse = scaler_y.inverse_transform(y_test_seq.reshape(-1, output_window_size))

            # Compute metrics for each step in the prediction window
            train_mse = np.mean([mean_squared_error(y_train_inverse[:, i], train_predictions_inverse[:, i]) for i in range(output_window_size)])
            test_mse = np.mean([mean_squared_error(y_test_inverse[:, i], test_predictions_inverse[:, i]) for i in range(output_window_size)])
            train_mae = np.mean([mean_absolute_error(y_train_inverse[:, i], train_predictions_inverse[:, i]) for i in range(output_window_size)])
            test_mae = np.mean([mean_absolute_error(y_test_inverse[:, i], test_predictions_inverse[:, i]) for i in range(output_window_size)])
            train_rmse = np.sqrt(train_mse)
            test_rmse = np.sqrt(test_mse)
            train_r2 = np.mean([r2_score(y_train_inverse[:, i], train_predictions_inverse[:, i]) for i in range(output_window_size)])
            test_r2 = np.mean([r2_score(y_test_inverse[:, i], test_predictions_inverse[:, i]) for i in range(output_window_size)])

            train_mape = np.mean([mean_absolute_percentage_error(y_train_inverse[:, i], train_predictions_inverse[:, i]) * 100 for i in range(output_window_size)])
            test_mape = np.mean([mean_absolute_percentage_error(y_test_inverse[:, i], test_predictions_inverse[:, i]) * 100 for i in range(output_window_size)])

            train_directional_acc = np.mean([np.mean(np.sign(y_train_inverse[1:, i] - y_train_inverse[:-1, i]) == np.sign(train_predictions_inverse[1:, i] - train_predictions_inverse[:-1, i])) for i in range(output_window_size)])
            test_directional_acc = np.mean([np.mean(np.sign(y_test_inverse[1:, i] - y_test_inverse[:-1, i]) == np.sign(test_predictions_inverse[1:, i] - test_predictions_inverse[:-1, i])) for i in range(output_window_size)])

        evaluation_time = time.time() - evaluation_start_time

        result = {
            "window_size": window_size,
            "output_window_size": output_window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "train_mse": train_mse,
            "test_mse": test_mse,
            "train_mae": train_mae,
            "test_mae": test_mae,
            "train_rmse": train_rmse,
            "test_rmse": test_rmse,
            "train_r2": train_r2,
            "test_r2": test_r2,
            "train_mape": train_mape,
            "test_mape": test_mape,
            "train_directional_acc": train_directional_acc,
            "test_directional_acc": test_directional_acc,
            "training_time": training_time,
            "evaluation_time": evaluation_time,
            "patience": patience,
            "activation_function": activation_function.__name__
        }

        with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=result.keys())
            if f.tell() == 0:
                writer.writeheader()
            writer.writerow(result)

        save_predictions(y_train_inverse, train_predictions_inverse, y_test_inverse, test_predictions_inverse, model_counter, file_prefix)

        if plot_loss:
            plot_and_save_loss(training_losses, validation_losses, file_prefix, model_counter)

        print(f"Results: Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
        print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
        print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

    except Exception as e:
        print(f"An error occurred: {e}")
        result = {
            "window_size": window_size,
            "output_window_size": output_window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "error": str(e)
        }
    finally:
        torch.cuda.empty_cache()

    return result

In [ ]:
parameter_space = {
    'window_size': [6],
    'prediction_steps': [6],
    'lstm_dim': [50, 80, 110, 140],
    'dense_dim': [50, 80, 110, 140],
    'num_layers': [1],
    'dropout': [0.0],
    'lr': [0.0001, 0.0002, 0.00005],
    'num_epochs': [50, 80, 110, 140, 170, 200],
    'batch_size': [32, 64, 96],
    'optimizer_type': ['Adam'],
    'patience': [24],
    'activation_function': [nn.Tanh]
}

In [ ]:
# Random search implementation
target_column = 'close_diff'
features = price_features + diff_features
file_prefix='LSTM_4h_future_randomsearch'

start = time.time()

# Set seed for reproducibility
random.seed(42)

# Generate random sample of parameter combinations
parameter_combinations = list(itertools.product(*parameter_space.values()))
random.shuffle(parameter_combinations)
random_sample = parameter_combinations[:100]

model_counter = 1

for params in random_sample:
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{100} with parameters: {param_dict}\n")
    
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, model_counter=model_counter, file_prefix=file_prefix, use_early_stopping=False, plot_loss=False)
    
    model_counter += 1

end = time.time()
total_time = end - start

print(f"Total time taken: {total_time} seconds")

In [ ]:
parameter_space = {
    'window_size': [6],
    'output_window_size': [6],
    'lstm_dim': [50, 80, 110, 140],
    'dense_dim': [50, 80, 110, 140],
    'num_layers': [1],
    'dropout': [0.0],
    'lr': [0.0001, 0.0002, 0.00005],
    'num_epochs': [50, 80, 110, 140, 170, 200],
    'batch_size': [32, 64, 96],
    'optimizer_type': ['Adam'],
    'patience': [24],
    'activation_function': [nn.Tanh]
}

In [ ]:
# Random search implementation
target_column = 'close_diff'
features = price_features + diff_features
file_prefix='LSTM_4h_future_randomsearch'

start = time.time()

# Set seed for reproducibility
random.seed(42)

# Generate random sample of parameter combinations
parameter_combinations = list(itertools.product(*parameter_space.values()))
random.shuffle(parameter_combinations)
random_sample = parameter_combinations[:100]

model_counter = 1

for params in random_sample:
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{100} with parameters: {param_dict}\n")
    
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, model_counter=model_counter, file_prefix=file_prefix, use_early_stopping=False, plot_loss=False)
    
    model_counter += 1

end = time.time()
total_time = end - start

print(f"Total time taken: {total_time} seconds")

In [ ]:
parameter_space = {
    'window_size': [48],
    'output_window_size': [48],
    'lstm_dim': [30, 50, 80, 110, 140],
    'dense_dim': [30, 50, 80, 110, 140],
    'num_layers': [1],
    'dropout': [0.0],
    'lr': [0.0001, 0.00005],
    'num_epochs': [50, 80, 110, 140, 170, 200],
    'batch_size': [10, 20, 30, 40],
    'optimizer_type': ['Adam'],
    'patience': [24],
    'activation_function': [nn.Tanh]
}

In [ ]:
# Random search implementation
target_column = 'close_diff'
features = price_features + diff_features
file_prefix='LSTM_30min_future_randomsearch'

start = time.time()

# Set seed for reproducibility
random.seed(42)

# Generate random sample of parameter combinations
parameter_combinations = list(itertools.product(*parameter_space.values()))
random.shuffle(parameter_combinations)
random_sample = parameter_combinations[:150]

model_counter = 1

for params in random_sample:
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{150} with parameters: {param_dict}\n")
    
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, model_counter=model_counter, file_prefix=file_prefix, use_early_stopping=False, plot_loss=False)
    
    model_counter += 1

end = time.time()
total_time = end - start

print(f"Total time taken: {total_time} seconds")